In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
features = 16
class LinearVAE(nn.Module):
    def __init__(self):
        super(LinearVAE, self).__init__()
        
        # Encoder
        self.enc_fc1 = torch.nn.Linear(in_features=28*28, out_features=512)
        self.enc_fc2 = torch.nn.Linear(in_features=512, out_features=features*2)
        
        # Decoder
        self.dec_fc1 = torch.nn.Linear(in_features=features*2, out_features=512)
        self.dec_fc2 = torch.nn.Linear(in_features=512, out_features=28*28)
        
        
    def reparameterize(self, mu, log_var):
        """
        Recieves log variance and mu. Outputs a sample.
        """
        var = torch.exp(0.5 * log_var) # TODO: why do we have the 0.5 term?
        eps = torch.randn_like(std) # Mean 0, Var 1
        sample = mu + eps * var # Gives us samples with mean MU and std VAR
        return sample
    
    
    def forward(self, x):
        x = F.relu(self.enc_fc1(x))
        x = F.relu(self.enc_fc2(x))
        
        # We now have some latent representation with shape [N, feature * 2]
        # We will reshape it to shape [-1, 2, feature], so it's now [N, 2, features]
        # Where [:, 0, :] is the mean, and [:, 1, :] is the variance
        
        mean = x[:, 0, :]
        var = x[:, 1, :]
        
        reparameterized = self.reparameterize(mean, var)
        
        reconstructed = F.relu(self.dec_fc1(reparameterized))
        reconstructed = torch.sigmoid(self.dec_fc2(reparameterized))
        return reconstruction, mu, log_var
        
        

In [7]:
import torch
import torchvision
import torch.optim as optim
import argparse
import matplotlib
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from tqdm import tqdm
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
matplotlib.style.use('ggplot')

In [12]:
# leanring parameters
epochs = 10
batch_size = 64
lr = 0.0001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

# train and validation data
train_data = datasets.MNIST(
    root='../input/data',
    train=True,
    download=True,
    transform=transform
)
val_data = datasets.MNIST(
    root='../input/data',
    train=False,
    download=True,
    transform=transform
)

100.1%

Extracting ../input/data/MNIST/raw/train-images-idx3-ubyte.gz


28.4%

0.5%5%

Extracting ../input/data/MNIST/raw/train-labels-idx1-ubyte.gz


100.4%

Extracting ../input/data/MNIST/raw/t10k-images-idx3-ubyte.gz


180.4%

Extracting ../input/data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [19]:
train_loader = DataLoader (
    train_data, 
    batch_size=batch_size,
    shuffle=True)

test_loader = DataLoader (
    val_data, 
    batch_size=batch_size,
    shuffle=False)

In [ ]:
model = LinearVAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
loss = nn.BCELoss(reduction="SUM")